# The InterlockLedger RESTful API

This notebook will show the usage of some features of the Python client of the InterlockLedger RESTful API.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

from il2_rest import RestNode

# Comment these two lines if you want to show InsecureRequestWarnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


## Getting an instance of a node

To use the `il2_rest` client, you need to create an instance of the `RestNode` by passing a certificate file and the address of the node (default address is `localhost`). 

> The PKCS12 certificate must be already imported to the InterlockLedger node and be permissioned on the desired chain. See the InterlockLedger node manual.

With the `RestNode` class, it is possible to retrieve details of the node, such as the list of valid apps in the network, peers, mirrors and chains.

In [ ]:
node = RestNode(cert_file = 'documenter.pfx', cert_pass = 'password', port = 32020)
print(node.details)

## Exploring the chains

To see and store records and documents, you need to use an instance of the `RestChain`. You can get `RestChain` instances by retrieving the list of chains in the network:

In [ ]:
chains = node.chains
for chain in chains :
    print(chain)

## Checking the content of a chain

By getting an instance of the `RestChain`, it is possible to retrieve and send information about the chain.

On this example, we will use the chain with the following id (`chain_id` - change this to your il2 chain).
It is possible to see the permitted apps and keys.
It is also possible to check the records stored in the chain.

In [ ]:
chain_id = 'A1wCG9hHhuVNb8hyOALHokYsWyTumHU0vRxtcK-iDKE'
chain = node.chain_by_id(chain_id)

print(chain.active_apps)

for key in chain.permitted_keys :
    print(key)

for record in chain.records_from(firstSerial=0, lastSerial=2) :
    print(record)

This chain has the app 4 active, so it is also possible to check and store documents directly.
Bellow we will be checking the list of the documents and getting the content of the documents using the `fileId`.

In [ ]:
for document in chain.documents :
    print(document)

document_bytes = chain.document_as_raw(fileId='WGWY2Xm-dTaLyu1IxZHv5WJBJFDI1lR5sBXMUVxyNwI#SHA256')
print(document_bytes)

## Managing Keys and Apps

If your are using a certificate allowed to permit key, you can permit other keys in the chain. The `permit_keys` method will return the list of permitted keys in the chain.

> To permit other keys, the certificate must be already imported to the Interlockledger node with actions for App #2 and actions 500,501.

In [ ]:
from il2_rest.models import KeyPermitModel
from il2_rest.enumerations import KeyPurpose
key_model = KeyPermitModel(app = 4, appActions = [1000, 1001], key_id = 'Key!MJ0kidltB324mfkiOG0aBlEocPA#SHA1',
                   name = 'documenter', publicKey = 'PubKey!KPgQEPgItqh4hu-oJtc7pSX0jp0fYEkV_kazPxOPGxiv1YX6dbt1QNb9AFb3mYpgUE9lRsehQz9Keh80K3mxdsURZbyhACRo3ljjKKBOQY4aKIIje9yPTTnJqg0XwwyBsx1zb-qEQaWm6S5HsVvMipGSfZIhgf3R2RYOvKR8zJRr7M1h7yoPN-02wzY1wubUpmpVB6aI_wAinTfUhBxKTuOkpe6M8OhPM-W4RUC-Et22Z4SzYK9-w08PULDBl3hCD2F-0K7TnQk8j-_1K0zV9bd2v0WovdjMrWUtMmWGcJ3Z2bJpB3-0e9Q_MxVw89r1nhYnj8zVf36HV8oVBZk4axWhFbTDrxADAQAB#RSA',
                   purposes = [KeyPurpose.Action, KeyPurpose.Protocol])

keys = chain.permit_keys([key_model])
for key in keys:
    print(key)

Similarly, you can permit apps using the following method. In this example we are trying to permit the app 4 (to store documents).

In [ ]:
apps = chain.permit_apps([4])
print(apps)

## Storing Records

It is possible to store documents using the model classes or using the unpacked method.

In [ ]:
from il2_rest.models import NewRecordModelAsJson
from il2_rest.models import NewRecordModel

model_json = NewRecordModelAsJson(applicationId = 1, payloadTagId = 300, rec_json= {'tagId': 300,'version' : 0, 'apps': [4]})
record_json = chain.add_record_as_json(model = model_json)
print(record_json)

model_bytes = NewRecordModel(applicationId = 1, payloadTagId = 300, payloadBytes = bytes([248, 52, 7, 5, 0, 0, 20, 2, 1, 4]))
record_bytes = chain.add_record(model_bytes)
print(record_bytes)

record_unpacked = chain.add_record_unpacked(applicationId = 1, payloadTagId = 300, rec_bytes = bytes([5, 0, 0, 20, 2, 1, 4]))
print(record_unpacked)

## Storing documents

There are three ways to store a document: plain text, bytes or file. To store a text document you can use the following script:


In [ ]:
from il2_rest.models import DocumentUploadModel

doc_resp = chain.store_document_from_text(content = 'Plain text', name = 'text_file.txt')
print(doc_resp)

doc_bytes = chain.store_document_from_bytes(doc_bytes = b'Bytes message!', name = 'bytes_file.txt', content_type = 'plain/text')
print(doc_bytes)

doc_model = DocumentUploadModel(name = 'other_bytes_file.txt', contentType = 'plain/text')
new_document = chain.store_document_from_bytes(doc_bytes = b'Other bytes message!', model = doc_model)
print(new_document)

doc_file = chain.store_document_from_file(file_path = './test.pdf', content_type = 'application/pdf')
print(doc_file)

## Interlockings

It is also possible to check or force interlocks using the API.

In [ ]:
from il2_rest.models import ForceInterlockModel

for interlock in chain.interlocks :
    print(interlock)

force_model = ForceInterlockModel(targetChain = '8fox30W54ZkzM-shfUeU5C7ad-_fsf5nICwNpkCUk5w')
interlock_model = chain.force_interlock(model = force_model)
print(interlock_model)